# BasicTick: Create Cluster: GW
This notebook will create (start) a Gateway cluster.

In [1]:
%%html
<style>
table {float:left}
</style>

## Setup

### Node Types
|Type|Mem (GB)|vCPUs|
|:---|---:|---:|
|kx.s.large|12|2|
|kx.s.xlarge|27|4|
|kx.s.2xlarge|54|8|
|kx.s.4xlarge|108|16|
|kx.s.8xlarge|216|32|
|kx.s.16xlarge|432|64|
|kx.s.32xlarge|864|128|


In [2]:
import os
import boto3
import json
import datetime

from managed_kx import *
from env_kdb_1 import *

# Cluster names and database
from basictick_setup import *

# Managed KX Database and Cluster names to create
SEC_THREADS='4'

# Cluster Settings
CODEBASE="basictick"
S3_CODE_PATH="code"

CAPACITY_CONFIG={ 'nodeCount': 1, 'nodeType': 'kx.s.2xlarge'}
CODE_CONFIG={ 's3Bucket': S3_BUCKET, 's3Key': f'{S3_CODE_PATH}/{CODEBASE}.zip' }
INIT_SCRIPT=f'{CODEBASE}/gwmkdbcluster.q'

CMD_ARGS=[
    { 'key': 's', 'value': SEC_THREADS }, 
    { 'key': 'codebase', 'value': CODEBASE}, 
    { 'key': 'rdb_name', 'value': RDB_CLUSTER_NAME}, 
    { 'key': 'hdb_name', 'value': HDB_CLUSTER_NAME}, 
]

In [3]:
# triggers credential get
session=None

try:
    # aws: use ada for credentials
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


# Deploy code

In [4]:
if AWS_ACCESS_KEY_ID is not None:
    cp = f"""
export AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY_ID}
export AWS_SECRET_ACCESS_KEY={AWS_SECRET_ACCESS_KEY}
export AWS_SESSION_TOKEN={AWS_SESSION_TOKEN}

zip -r -X {CODEBASE}.zip {CODEBASE}    
aws s3 cp {CODEBASE}.zip s3://{S3_BUCKET}/{S3_CODE_PATH}/{CODEBASE}.zip
aws s3 ls {S3_BUCKET}/{S3_CODE_PATH}/
"""
else:
    cp = f"""
zip -r -X {CODEBASE}.zip {CODEBASE}    
aws s3 cp {CODEBASE}.zip s3://{S3_BUCKET}/{S3_CODE_PATH}/{CODEBASE}.zip
aws s3 ls {S3_BUCKET}/{S3_CODE_PATH}/
"""
    
# execute the S3 copy
os.system(cp)

updating: basictick/ (stored 0%)
updating: basictick/connectmkdb.q (deflated 63%)
updating: basictick/example.schema.q (deflated 12%)
updating: basictick/feedmkdb.q (deflated 53%)
updating: basictick/funcDownHandle.q (deflated 33%)
updating: basictick/gwmkdbcluster.q (deflated 61%)
updating: basictick/hdbmkdb.q (deflated 52%)
updating: basictick/loadDep.q (deflated 57%)
updating: basictick/query.q (deflated 19%)
updating: basictick/rdbmkdb.q (deflated 57%)
updating: basictick/tickerplant2 (stored 0%)
updating: basictick/tickerplant3 (stored 0%)
updating: basictick/tp.q (deflated 52%)
updating: basictick/.ipynb_checkpoints/ (stored 0%)
updating: basictick/.ipynb_checkpoints/tickerplant3-checkpoint (stored 0%)
updating: basictick/.ipynb_checkpoints/tickerplant-checkpoint (stored 0%)
updating: basictick/tickerplant (stored 0%)
updating: basictick/.ipynb_checkpoints/rdbmkdb-checkpoint.q (deflated 57%)
updating: basictick/.ipynb_checkpoints/gwmkdbcluster-checkpoint.q (deflated 61%)
updating

0

## Create Cluster

In [5]:
# be sure the HDB and RDB are running
wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=HDB_CLUSTER_NAME)
wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=RDB_CLUSTER_NAME)

print("*** HDB and RDB are running ***")

*** HDB and RDB are running ***


In [6]:
resp = client.create_kx_cluster(
    environmentId=ENV_ID, 
    clusterName=GW_CLUSTER_NAME,
    clusterType='GATEWAY',
    releaseLabel = '1.0',
    capacityConfiguration=CAPACITY_CONFIG,
    clusterDescription="Created with create_GW notebook",
    executionRole=EXECUTION_ROLE,
    code=CODE_CONFIG,
    initializationScript=INIT_SCRIPT,
    commandLineArguments=CMD_ARGS,
    azMode=AZ_MODE,
    availabilityZoneId=AZ_ID,
    vpcConfiguration={ 
        'vpcId': VPC_ID,
        'securityGroupIds': SECURITY_GROUPS,
        'subnetIds': SUBNET_IDS,
        'ipAddressType': 'IP_V4' }
)

In [7]:
resp

{'ResponseMetadata': {'RequestId': 'a3fc3f60-b81b-46df-8948-b2b281fdd517',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '1060',
   'connection': 'keep-alive',
   'date': 'Tue, 25 Jul 2023 22:42:54 GMT',
   'x-amzn-requestid': 'a3fc3f60-b81b-46df-8948-b2b281fdd517',
   'x-amz-apigw-id': 'IpFstHukoAMF-BQ=',
   'x-amzn-trace-id': 'Root=1-64c04fea-5d581ed46a18bf5d687439b3',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 3d3fd40be4e4bfdd1e1bebf86df63a76.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD55-P1',
   'x-amz-cf-id': 'xfibqaK-Kvqt5bTK9AtmmjDUIDBEsudtG-LwBgG3KMnIUu4s0FUqVA=='},
  'RetryAttempts': 0},
 'status': 'PENDING',
 'clusterName': 'GATEWAY_basictickdb',
 'clusterType': 'GATEWAY',
 'clusterDescription': 'Created with create_GW notebook',
 'capacityConfiguration': {'nodeType': 'kx.s.2xlarge', 'nodeCount': 1},
 'releaseLabel': '1.0',
 'vpcConfiguration': {'vpcId': 'vpc-0fe2b9c50f3ad382f',
  'securityGroupIds': ['s

In [8]:
wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=GW_CLUSTER_NAME)
print("** DONE **")

** DONE **


In [9]:
cdf = get_clusters(client, environmentId=ENV_ID, clusterType="GATEWAY")

display(cdf)

,clusterName,status,clusterType,capacityConfiguration,commandLineArguments,clusterDescription,lastModifiedTimestamp,createdTimestamp,databaseName
0,GATEWAY_basictickdb,RUNNING,GATEWAY,"{'nodeType': 'kx.s.2xlarge', 'nodeCount': 1}","[{'key': 's', 'value': '4'}, {'key': 'codebase', 'value': 'basictick'}, {'key': 'rdb_name', 'value': 'RDB_basictickdb'}, {'key': 'hdb_name', 'value': 'HDB_basictickdb'}]",Created with create_GW notebook,2023-07-25 22:55:43.727000+00:00,2023-07-25 22:42:54.070000+00:00,None


In [10]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2023-07-25 22:56:05.137244
